In [6]:
"""
    @Author: Vipul Shete
    @Date: 2022-12-24
    @Last Modified by: Vipul Shete
    @Last Modified Date: 2022-12-28
    @Title : Stock Market Details and company overview (UC-2).
"""

'\n    @Author: Vipul Shete\n    @Date: 2022-12-24\n    @Last Modified by: Vipul Shete\n    @Last Modified Date: 2022-12-28\n    @Title : Stock Market Details and company overview (UC-2).\n'

In [4]:
from alpha_vantage.timeseries import TimeSeries
import pandas as pd
from pyspark.sql import SparkSession
import requests

In [5]:
spark = SparkSession.builder\
                    .master("local")\
                    .appName('stock_market_UC1')\
                    .getOrCreate()

22/12/28 10:39:27 WARN Utils: Your hostname, vipul-HP-Pavilion-Laptop-15-cs2xxx resolves to a loopback address: 127.0.1.1; using 192.168.43.246 instead (on interface wlx001ea63c098f)
22/12/28 10:39:27 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/12/28 10:39:28 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
spark

In [9]:
def daily_stock_price(company_name, date):
    """
        Description:
            Get the stock daily status using alpha vantage (open , high, low, close values)
        Parameter:
            company_name: name of company (symbol used in alpha vantage docomenation)
            date: date of records.
        Return: 
            stock details open, high, low, close 
    """
    url = f"https://www.alphavantage.co/query?function=TIME_SERIES_DAILY_ADJUSTED&symbol={company_name}&apikey=3ZN85270ZOKUBGMP"
    r = requests.get(url)
    data = r.json()
    date_stock_price = data["Time Series (Daily)"][date]
    
    open = date_stock_price["1. open"]
    high = date_stock_price["2. high"]
    low = date_stock_price["3. low"]
    close = date_stock_price["4. close"]
    
    return open, high, low, close

def company_details(company_name, date):
    """
        Description:
            make a dataframe of single company
        Parameter:
            company_name: name of company (symbol used in alpha vantage docomenation)
            date: date of records.
        Return: 
            dataframe 
    """
    open, high, low, close = daily_stock_price(company_name, date)

    columns = ["Company_name", "Open", "High", "Low", "Close"]
    values = [(company_name, open, high, low, close)]

    stock_info = spark.createDataFrame(values, columns)
    
    return stock_info

In [9]:
def company_overview(company_name):
    """
        Description:
            Get the company overview using alpha vantage (pe_ratio and gross_profit)
        Parameter:
            company_name: name of company (symbol used in alpha vantage docomenation)
            date: date of records.
        Return: 
            stock overview pe_ratio, gross_profit
    """
    url = f"https://www.alphavantage.co/query?function=OVERVIEW&symbol={company_name}&apikey=3ZN85270ZOKUBGMP"
    r = requests.get(url)
    data = r.json()
    
    pe_ratio = data["PERatio"]
    gross_profit = data["GrossProfitTTM"]

    return pe_ratio, gross_profit
    

In [10]:
company_overview("IBM")

{'Symbol': 'IBM', 'AssetType': 'Common Stock', 'Name': 'International Business Machines', 'Description': 'International Business Machines Corporation (IBM) is an American multinational technology company headquartered in Armonk, New York, with operations in over 170 countries. The company began in 1911, founded in Endicott, New York, as the Computing-Tabulating-Recording Company (CTR) and was renamed International Business Machines in 1924. IBM is incorporated in New York. IBM produces and sells computer hardware, middleware and software, and provides hosting and consulting services in areas ranging from mainframe computers to nanotechnology. IBM is also a major research organization, holding the record for most annual U.S. patents generated by a business (as of 2020) for 28 consecutive years. Inventions by IBM include the automated teller machine (ATM), the floppy disk, the hard disk drive, the magnetic stripe card, the relational database, the SQL programming language, the UPC barcod

('23.26', '31486000000')

In [10]:
company_name = "APLE"
date = "2022-12-23"
apple = company_details(company_name, date)
apple.show()

+------------+-----+-----+------+-----+
|Company_name| Open| High|   Low|Close|
+------------+-----+-----+------+-----+
|        APLE|15.57|15.86|15.465|15.83|
+------------+-----+-----+------+-----+



In [11]:
company_name = "DPZ"
date = "2022-12-23"
dominos_pizza = company_details(company_name, date)
dominos_pizza.show()

+------------+------+------+-----+------+
|Company_name|  Open|  High|  Low| Close|
+------------+------+------+-----+------+
|         DPZ|351.35|354.34|347.0|353.02|
+------------+------+------+-----+------+



In [12]:
company_name = "MCD"
date = "2022-12-23"
mc_donalds = company_details(company_name, date)
mc_donalds.show()

+------------+-----+-------+------+------+
|Company_name| Open|   High|   Low| Close|
+------------+-----+-------+------+------+
|         MCD|265.9|268.345|264.79|267.57|
+------------+-----+-------+------+------+



In [13]:
company_name = "ACN"
date = "2022-12-23"
accenture = company_details(company_name, date)
accenture.show()

+------------+------+------+--------+------+
|Company_name|  Open|  High|     Low| Close|
+------------+------+------+--------+------+
|         ACN|265.79|266.71|263.3079|266.09|
+------------+------+------+--------+------+



In [14]:
company_name = "TCS"
date = "2022-12-23"
tata_con_services = company_details(company_name, date)
tata_con_services.show()

+------------+----+----+------+-----+
|Company_name|Open|High|   Low|Close|
+------------+----+----+------+-----+
|         TCS|4.18|4.32|4.1514| 4.26|
+------------+----+----+------+-----+



## merage data frame

In [15]:
appended_one = apple.union(dominos_pizza)
appended_two = appended_one.union(mc_donalds)
appended_three = appended_two.union(accenture)
company_details = appended_three.union(tata_con_services)

In [16]:
company_details.show()

+------------+------+-------+--------+------+
|Company_name|  Open|   High|     Low| Close|
+------------+------+-------+--------+------+
|        APLE| 15.57|  15.86|  15.465| 15.83|
|         DPZ|351.35| 354.34|   347.0|353.02|
|         MCD| 265.9|268.345|  264.79|267.57|
|         ACN|265.79| 266.71|263.3079|266.09|
|         TCS|  4.18|   4.32|  4.1514|  4.26|
+------------+------+-------+--------+------+

